### Sequence pre-processing

This notebook will have a csv containing the six input streams, gyro and acc in the three axis. Output will be a tensor from tensorflow which is 6xY where Y will be the selected length of the sequence. The model we use can only use selected data timeframes as input and this notebook aims to prepare the CSV for this use. 
This notebook has two parameters; overlap, a float between 0 and 1 which indicates the overlap between two time frames. The second is length, this is an integer which contains the desired length of the sequence.  

In [4]:
from pathlib import Path
from config import config

In [ ]:
def create_sequence(metadata_csv_path: Path, overlap: float, length: int):
    data_point = length * 200 # GoPro Max 360 has a sample speed of 200/s
    
    

In [7]:
import pandas as pd
from pathlib import Path

def load_csv_to_dataframe(file_path, delimiter=',', encoding='utf-8'):
    """
    Load a CSV file into a pandas DataFrame.

    Parameters:
        file_path (Path or str): The path to the CSV file as a Path object or string.
        delimiter (str): The delimiter used in the CSV file. Default is ','.
        encoding (str): The encoding of the CSV file. Default is 'utf-8'.

    Returns:
        pd.DataFrame: The loaded pandas DataFrame.
    """
    try:
        # Ensure file_path is a string if it's a Path object
        if isinstance(file_path, Path):
            file_path = file_path.as_posix()
        
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path, delimiter=delimiter, encoding=encoding)
        print("CSV file successfully loaded into a DataFrame.")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Please check the path.")
    except Exception as e:
        print(f"An error occurred while loading the CSV: {e}")

# Example usage
if __name__ == "__main__":
    # Replace with your Path object pointing to the CSV file
    file_path = Path("your_file.csv")
    dataframe = load_csv_to_dataframe(file_path)

    # Display the first few rows if the file was loaded successfully
    if dataframe is not None:
        print(dataframe.head())


ModuleNotFoundError: No module named 'pandas'

In [6]:
create_sequence(config.DATA_DIR / 'ÇSVs' / 'GH010038-ACC&GYRO.csv', 0, 10)
import venv

2000
